original: https://www.guruguru.science/competitions/25/discussions/8b97734b-1f76-4075-b1af-5d227d6b70e8/ (@yururoi)


In [1]:
!rm -r /kaggle/working/*
%cd /kaggle/working

/kaggle/working


In [2]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [3]:
import yaml
from penguinml.utils.logger import get_logger, init_logger
from penguinml.utils.set_seed import seed_base

MODEL_NAME = "lightgbm"
CFG = yaml.safe_load(open(os.path.join(PACKAGE_DIR, "config.yaml"), "r"))
print(CFG[MODEL_NAME]["execution"]["exp_id"])
CFG["output_dir"] = f"/kaggle/output/{CFG[MODEL_NAME]['execution']['exp_id']}"
!rm -r {CFG["output_dir"]}
os.makedirs(CFG["output_dir"], exist_ok=True)

init_logger(f"{ CFG[MODEL_NAME]['execution']['exp_id']}.log")
logger = get_logger("main")
seed_base(CFG[MODEL_NAME]["execution"]["seed"])

2024-11-21 00:42:32.812150: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 00:42:32.837579: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


exp_018


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
set seed: 46


In [4]:
import glob
import os
import pickle
import warnings
from pathlib import Path

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


In [5]:
class Config:
    N_FOLD = 5
    RANDOM_SATE = 42


NB = "exp1015"

In [6]:
ROOT_DIR = Path("/kaggle")
DATA_DIR = ROOT_DIR / Path("input/atmaCup#18_dataset")

In [7]:
train_df = pl.read_csv(DATA_DIR / "train_features.csv")
test_df = pl.read_csv(DATA_DIR / "test_features.csv")

## 特徴量生成


In [8]:
# train_df と test_dfを結合（特徴量エンジニアリングをしやすくするため）
_all_df = pl.concat([train_df, test_df], how="diagonal")

In [9]:
agg_cols = [
    "vEgo",
    "aEgo",
    "steeringAngleDeg",
    "steeringTorque",
    "gas",
]  # 同一シーンから集計する値のカラム名

# 同一シーンから特徴量作成
exprs = []
exprs += [
    pl.col(agg_col).shift(-1).over("scene").alias(f"{agg_col}_shift-1") for agg_col in agg_cols
]  # 1ステップ前の時間の値
exprs += [
    pl.col(agg_col).shift(1).over("scene").alias(f"{agg_col}_shift1") for agg_col in agg_cols
]  # 1ステップ後の時間の値
exprs += [
    pl.col(agg_col).diff(-1).over("scene").alias(f"{agg_col}_diff-1") for agg_col in agg_cols
]  # 1ステップ前の時間の値との差分
exprs += [
    pl.col(agg_col).diff(1).over("scene").alias(f"{agg_col}_diff1") for agg_col in agg_cols
]  # 1ステップ後の時間の値との差分
exprs += [pl.col(agg_col).mean().over("scene").alias(f"{agg_col}_mean") for agg_col in agg_cols]  # 同一シーンの平均値
exprs += [pl.col(agg_col).std().over("scene").alias(f"{agg_col}_std") for agg_col in agg_cols]  # 同一シーンの標準偏差
exprs += [pl.col(agg_col).max().over("scene").alias(f"{agg_col}_max") for agg_col in agg_cols]  # 同一シーンの最大値
exprs += [pl.col(agg_col).min().over("scene").alias(f"{agg_col}_min") for agg_col in agg_cols]  # 同一シーンの最小値

_all_df = (
    _all_df.with_columns(
        # ID からシーンとデシ秒を作成
        pl.col("ID").str.split("_").list.get(0).alias("scene"),
        pl.col("ID").str.split("_").list.get(1).cast(pl.Int32).alias("decisecond"),
    )
    .sort(
        # shiftと diffが時系列順に並んでいる必要があるためシーンごとに時間軸でソート
        "scene",
        "decisecond",
    )
    .with_columns(exprs)
)

In [10]:
train_folds = pl.read_csv(CFG["dataset"]["train_fold_path"]).rename({"sceneID": "scene"})
_all_df = _all_df.join(train_folds, how="left", on="scene")
# assert train_df["fold"].null_count() == 0

In [11]:
# YOLOの検出結果
import json

yolo_paths = glob.glob("/kaggle/input/yolo-det/det/*.json")
yolo_dfs = []
for path in tqdm(yolo_paths):
    ID = os.path.basename(path).split(".")[0]
    with open(path, "r") as f:
        data = json.load(f)

    yolo_feature = {
        "ID": ID,
        "num_objects": len(data),
    }
    for bbox in data:
        if bbox["x1"] == bbox["x2"] or bbox["y1"] == bbox["y2"]:
            continue

        if bbox["cls"] != "car":
            continue

        # count
        if bbox["cls"] not in yolo_feature:
            yolo_feature[bbox["cls"]] = 0
        yolo_feature[bbox["cls"]] += 1

        # 最も横方向が中央にあるものの情報
        if f"center_x_{bbox['cls']}" not in yolo_feature:
            yolo_feature[f"center_x_{bbox['cls']}"] = -1
        current_dist = abs(yolo_feature[f"center_x_{bbox['cls']}"] - 64)
        now_center_x = (bbox["x1"] + bbox["x2"]) / 2
        now_dist = abs(now_center_x - 64)
        if now_dist < current_dist:
            yolo_feature[f"center_x_{bbox['cls']}"] = now_center_x
            yolo_feature[f"center_y_{bbox['cls']}"] = (bbox["y1"] + bbox["y2"]) / 2
            yolo_feature[f"width_{bbox['cls']}"] = bbox["x2"] - bbox["x1"]
            yolo_feature[f"height_{bbox['cls']}"] = bbox["y2"] - bbox["y1"]
            yolo_feature[f"bottom_{bbox['cls']}"] = bbox["y2"]
            yolo_feature[f"area_{bbox['cls']}"] = (bbox["x2"] - bbox["x1"]) * (bbox["y2"] - bbox["y1"])
            yolo_feature[f"aspect_ratio_{bbox['cls']}"] = (bbox["x2"] - bbox["x1"]) / (bbox["y2"] - bbox["y1"])
            yolo_feature[f"conf_{bbox['cls']}"] = bbox["conf"]
    yolo_dfs.append(yolo_feature)
yolo_df = pl.DataFrame(yolo_dfs)

_all_df = _all_df.join(yolo_df, how="left", on="ID")

100%|██████████| 45098/45098 [00:00<00:00, 77790.74it/s]


In [12]:
# depth
import cv2

depth_features = []
for ID in tqdm(_all_df["ID"]):
    path = f"/kaggle/input/depth_image/depth/{ID}/0.png"
    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    patch_size = 16
    this_features = {"ID": ID}
    for i in range(0, 128, patch_size):
        for j in range(0, 64, patch_size):
            patch = image[i : i + patch_size, j : j + patch_size]
            this_features[f"patch_{i}_{j}_mean"] = np.mean(patch)
            this_features[f"patch_{i}_{j}_median"] = np.median(patch)
    depth_features.append(this_features)
depth_df = pl.DataFrame(depth_features)
_all_df = _all_df.join(depth_df, how="left", on="ID")

  0%|          | 0/45098 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 45098/45098 [00:18<00:00, 2446.90it/s]


## feature and target


In [13]:
targets = [
    "x_0",
    "y_0",
    "z_0",
    "x_1",
    "y_1",
    "z_1",
    "x_2",
    "y_2",
    "z_2",
    "x_3",
    "y_3",
    "z_3",
    "x_4",
    "y_4",
    "z_4",
    "x_5",
    "y_5",
    "z_5",
]

# 使う特徴量を指定するより使わない特徴量を指定するほうが試行錯誤が楽
del_columns = targets + ["ID", "scene", "gearShifter", "fold"]

features = list(set(_all_df.columns) - set(del_columns))
features.sort()

print(features)

['aEgo', 'aEgo_diff-1', 'aEgo_diff1', 'aEgo_max', 'aEgo_mean', 'aEgo_min', 'aEgo_shift-1', 'aEgo_shift1', 'aEgo_std', 'area_car', 'aspect_ratio_car', 'bottom_car', 'brake', 'brakePressed', 'car', 'center_x_car', 'center_y_car', 'conf_car', 'decisecond', 'gas', 'gasPressed', 'gas_diff-1', 'gas_diff1', 'gas_max', 'gas_mean', 'gas_min', 'gas_shift-1', 'gas_shift1', 'gas_std', 'height_car', 'leftBlinker', 'num_objects', 'patch_0_0_mean', 'patch_0_0_median', 'patch_0_16_mean', 'patch_0_16_median', 'patch_0_32_mean', 'patch_0_32_median', 'patch_0_48_mean', 'patch_0_48_median', 'patch_112_0_mean', 'patch_112_0_median', 'patch_112_16_mean', 'patch_112_16_median', 'patch_112_32_mean', 'patch_112_32_median', 'patch_112_48_mean', 'patch_112_48_median', 'patch_16_0_mean', 'patch_16_0_median', 'patch_16_16_mean', 'patch_16_16_median', 'patch_16_32_mean', 'patch_16_32_median', 'patch_16_48_mean', 'patch_16_48_median', 'patch_32_0_mean', 'patch_32_0_median', 'patch_32_16_mean', 'patch_32_16_median', 

In [14]:
# MAEを計算
def evaluation(true_values, pred_values):
    abs_diff = abs(true_values - pred_values)
    mae = np.mean(
        abs_diff.reshape(
            -1,
        )
    )
    return mae

### encoding


In [15]:
# label encdoding
categorical_columns = ["gearShifter"]

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    _all_df = _all_df.with_columns(pl.Series(le.fit_transform(_all_df[col])).alias(f"{col}_le"))
cate_features = [f"{col}_le" for c in categorical_columns]
features = list(set(features) | set(cate_features))

# count encoding
count_enc = ["gearShifter"]
_all_df = _all_df.with_columns([pl.col(c).count().over(c).alias(f"{c}_count") for c in count_enc])
count_features = [f"{c}_count" for c in count_enc]
features = list(set(features) | set(count_features))


train_df = train_df.join(_all_df, how="left", on="ID")
test_df = test_df.join(_all_df, how="left", on="ID")

In [16]:
def train_lgbm(target, params):
    oof_pred = np.zeros(len(train_df))
    y_pred = np.zeros(len(test_df))
    models = []
    cv_scores = {}

    for fold in range(5):
        # print(f"fold{fold}: ", end="")

        # TrainとTestに分割
        x_train = train_df.filter(pl.col("fold") != fold).select(features)
        x_val = train_df.filter(pl.col("fold") == fold).select(features)
        y_train = train_df.filter(pl.col("fold") != fold).select(target)
        y_val = train_df.filter(pl.col("fold") == fold).select(target)

        test = test_df[features]

        # create Dataset
        train_set = lgb.Dataset(
            x_train.to_pandas(),
            y_train.to_pandas(),
            categorical_feature=cate_features,
            free_raw_data=False,
        )
        val_set = lgb.Dataset(
            x_val.to_pandas(),
            y_val.to_pandas(),
            categorical_feature=cate_features,
            free_raw_data=False,
        )

        # train
        model = lgb.train(
            params,
            train_set,
            valid_sets=[train_set, val_set],
            num_boost_round=10000,
            callbacks=[
                lgb.early_stopping(stopping_rounds=100, verbose=False),
                lgb.log_evaluation(500000),
            ],
        )

        models.append(model)

        fold_pred = model.predict(x_val.to_pandas())

        score = evaluation(y_val.to_numpy().reshape(-1), fold_pred)
        cv_scores[f"cv{fold}"] = score

        # oof_pred[test_index] = fold_pred
        oof_pred[train_df["fold"].to_numpy() == fold] = fold_pred

        y_pred += model.predict(test.to_pandas()) / Config.N_FOLD

        # print(f"{score}")

    oof_score = evaluation(train_df[target].to_numpy().reshape(-1), oof_pred)
    # print(f"OOF score is {oof_score}")

    return oof_pred, y_pred, models

## 学習


In [17]:
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# def add_dt_features(train: pl.DataFrame):
#     """dt秒後の特徴"""
#     train = train.with_columns(
#         # vt
#         (pl.col("vEgo") * pl.col("dt").cast(pl.Float32)).alias("linear_movement@dt"),
#         # vt + 0.5at^2
#         ((pl.col("vEgo") + 0.5 * pl.col("aEgo") * pl.col("dt").cast(pl.Float32) ** 2).alias("movement@dt")),
#         # v + at
#         (pl.col("vEgo") + pl.col("aEgo") * pl.col("dt").cast(pl.Float32)).alias("velocity@dt"),
#     )
#     return train

import optuna


def objective(trial):
    max_depth = trial.suggest_int("max_depth", 3, 8)
    num_leaves = int(0.7 * max_depth**2)
    colsample_bytree = trial.suggest_uniform("colsample_bytree", 0.2, 1.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-3, 1e3)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-3, 1e3)
    subsample = trial.suggest_loguniform("subsample", 0.5, 1.0)
    min_child_samples = trial.suggest_int("min_child_samples", 5, 400)
    subsample_freq = trial.suggest_int("subsample_freq", 0, 6)

    params = {
        "boosting_type": "gbdt",
        "metric": "mae",  # 今回の評価指標がMAEを使用
        "objective": "regression",
        "n_jobs": -1,
        "seed": Config.RANDOM_SATE,
        "learning_rate": 0.01,
        # "device": "gpu"
        "verbosity": -1,
        "max_depth": max_depth,
        "num_leaves": num_leaves,
        "colsample_bytree": colsample_bytree,
        "reg_alpha": reg_alpha,
        "reg_lambda": reg_lambda,
        "subsample": subsample,
        "min_child_samples": min_child_samples,
        "subsample_freq": subsample_freq,
        # "n_estimators": 10,
    }

    models_dict = {}
    test_pred = []
    oof_pred = []
    for target in targets:
        # print("=" * 50)
        # print(f"# {target}")
        # print("=" * 50)

        # # dt features
        # dt = float(target.split("_")[-1]) * 0.5 + 0.5
        # train_df = train_df.with_columns(pl.lit(dt).alias("dt"))
        # test_df = test_df.with_columns(pl.lit(dt).alias("dt"))
        # train_df = add_dt_features(train_df)
        # test_df = add_dt_features(test_df)
        # features = list(set(features) | set(["linear_movement@dt", "movement@dt", "velocity@dt"]))

        oof_preds_partial, y_pred_partial, models_partial = train_lgbm(target, params)
        models_dict[target] = models_partial
        oof_pred.append(oof_preds_partial)
        test_pred.append(y_pred_partial)
    mae = evaluation(train_df[targets].to_numpy(), np.vstack(oof_pred).T)
    return mae


study = optuna.create_study(direction="minimize")
study.enqueue_trial(
    {
        "max_depth": 7,
        "colsample_bytree": 1.0,
        "reg_alpha": 1e-3,
        "reg_lambda": 1e-3,
        "subsample": 1.0,
        "min_child_samples": 20,
        "subsample_freq": 0,
    }
)
study.optimize(objective, n_trials=200)

[I 2024-11-21 00:42:55,809] A new study created in memory with name: no-name-23ae72dd-288e-4717-ba00-63d277cb3280
[I 2024-11-21 00:49:01,224] Trial 0 finished with value: 0.20889926848570828 and parameters: {'max_depth': 7, 'colsample_bytree': 1.0, 'reg_alpha': 0.001, 'reg_lambda': 0.001, 'subsample': 1.0, 'min_child_samples': 20, 'subsample_freq': 0}. Best is trial 0 with value: 0.20889926848570828.
[I 2024-11-21 00:58:00,605] Trial 1 finished with value: 0.2257071692103231 and parameters: {'max_depth': 5, 'colsample_bytree': 0.6654388599971016, 'reg_alpha': 0.023540330983112142, 'reg_lambda': 162.32735309201252, 'subsample': 0.5636625122102001, 'min_child_samples': 202, 'subsample_freq': 5}. Best is trial 0 with value: 0.20889926848570828.
[I 2024-11-21 01:05:58,588] Trial 2 finished with value: 0.22426596506962568 and parameters: {'max_depth': 3, 'colsample_bytree': 0.6578180982869597, 'reg_alpha': 0.07221556640229311, 'reg_lambda': 14.745916171907252, 'subsample': 0.669137886025491

In [ ]:
df = study.trials_dataframe()
best_params = study.best_params
best_score = study.best_value

# submit ファイル作成


In [ ]:
# sub_col_names = [
#     "x_0",
#     "y_0",
#     "z_0",
#     "x_1",
#     "y_1",
#     "z_1",
#     "x_2",
#     "y_2",
#     "z_2",
#     "x_3",
#     "y_3",
#     "z_3",
#     "x_4",
#     "y_4",
#     "z_4",
#     "x_5",
#     "y_5",
#     "z_5",
# ]
# sub_df = pl.DataFrame(np.vstack(test_pred).T, schema=sub_col_names)
# sub_df = sub_df.with_columns(pl.Series("ID", test_df["ID"]))
# sub_df.write_csv(os.path.join(CFG["output_dir"], "submission.csv"))

# oof_df = pl.DataFrame(np.vstack(oof_pred).T, schema=sub_col_names)
# oof_df = oof_df.with_columns(pl.Series("ID", train_df["ID"]))
# oof_df.write_csv(os.path.join(CFG["output_dir"], "oof.csv"))

# display(sub_df)
# sub_df.drop("ID").write_csv(os.path.join(CFG["output_dir"], "submission_wo_ID.csv"))


x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,ID
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
1.451966,-0.05001,0.000828,3.02292,-0.118906,0.003799,4.581418,-0.226131,0.005058,5.960247,-0.197664,0.001139,7.293397,-0.129997,0.012701,8.633616,0.008382,0.020247,"""012baccc145d400c896cb82065a93d…"
0.949345,0.37931,0.002446,1.78144,0.997347,0.009869,2.416597,1.656721,0.016779,3.018493,2.455365,0.022162,3.717404,3.277484,0.041375,4.531961,4.347581,0.051499,"""012baccc145d400c896cb82065a93d…"
1.574644,0.012977,-0.000396,3.284368,0.013113,0.001907,4.818523,0.027939,0.00021,6.35423,0.058121,-0.007488,7.929633,0.062045,-0.002237,9.17903,0.122829,-0.008452,"""012baccc145d400c896cb82065a93d…"
0.834858,0.071709,-0.004797,1.669115,0.265695,-0.014526,2.469084,0.648576,-0.019613,2.996955,1.083865,-0.037253,3.617305,1.894096,-0.054112,4.219986,2.808786,-0.082212,"""012baccc145d400c896cb82065a93d…"
0.819024,0.003951,-0.004481,1.425394,0.007575,-0.009175,1.895713,0.001957,-0.018596,2.230749,-0.009298,-0.03604,2.186012,-0.010073,-0.043591,1.669718,-0.026504,-0.049678,"""01d738e799d260a10f6324f78023b3…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
6.537135,0.008542,0.014563,13.798478,0.041535,0.036641,21.022014,0.096245,0.052083,28.292109,0.186007,0.07237,35.626491,0.305797,0.084002,43.041352,0.443229,0.090601,"""ff4f00a76fbf4db0cb15579c7c6086…"
6.988506,0.000371,-0.005645,14.857086,-0.016367,-0.041432,22.885263,-0.067638,-0.103802,31.022227,-0.134661,-0.222285,39.254969,-0.227326,-0.368399,47.522269,-0.336563,-0.55295,"""ff4f00a76fbf4db0cb15579c7c6086…"
7.413827,-0.000097,0.016546,15.655867,-0.016144,0.04354,23.843606,-0.041808,0.073727,31.945971,-0.068803,0.117456,39.950944,-0.119747,0.172923,47.864668,-0.202397,0.236791,"""ff4f00a76fbf4db0cb15579c7c6086…"
